In [3]:
import numpy as np

In [4]:
# 載入資料
X_train = np.load('X_train.npy')
X_val = np.load('X_val.npy')
X_test = np.load('X_test.npy')
y_train = np.load('y_train.npy')
y_val = np.load('y_val.npy')
y_test = np.load('y_test.npy')

In [5]:
# 重塑數據形狀為 (樣本數, 高度, 寬度, 頻道數)
X_train = X_train[..., np.newaxis]
X_val = X_val[..., np.newaxis]
X_test = X_test[..., np.newaxis]

print(X_train.shape)
print(X_val.shape)
print(X_test.shape)

(5328, 128, 1379, 1)
(666, 128, 1379, 1)
(666, 128, 1379, 1)


In [6]:
import tensorflow as tf
from keras import Input
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, GlobalAveragePooling2D

def create_model(input_shape):
    input_img = Input(shape=input_shape)
    x = Conv2D(32, kernel_size=(3, 3), activation='relu', padding='same')(input_img)
    x = MaxPooling2D((2, 2), padding='same')(x)
    x = Conv2D(64, kernel_size=(3, 3), activation='relu', padding='same')(x)
    x = MaxPooling2D((2, 2), padding='same')(x)
    x = Conv2D(64, kernel_size=(3, 3), activation='relu', padding='same')(x)
    x = MaxPooling2D((2, 2), padding='same')(x)
    x = Dropout(0.25)(x)
    x = GlobalAveragePooling2D()(x)
    x = Dense(64, activation='relu')(x)
    x = Dropout(0.5)(x)
    output = Dense(1, activation='sigmoid')(x)
    model = Model(inputs=input_img, outputs=output)
    return model

In [7]:
input_shape = np.shape(X_train)[1:]
model = create_model(input_shape)
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 128, 1379, 1)]    0         
                                                                 
 conv2d (Conv2D)             (None, 128, 1379, 32)     320       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 64, 690, 32)      0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 64, 690, 64)       18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 32, 345, 64)      0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 32, 345, 64)       36928 

# 編譯模型

In [8]:
from keras.optimizers import Adam

initial_learning_rate = 0.001

# 創建 Adam 優化器並指定學習率
optimizer = Adam(learning_rate=initial_learning_rate)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

# 訓練模型

In [9]:
history = model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=50, batch_size=16)

Epoch 1/50
333/333 [==============================] - 110s 238ms/step - loss: 0.0000e+00 - accuracy: 0.5017 - val_loss: 0.0000e+00 - val_accuracy: 0.4985
Epoch 2/50
333/333 [==============================] - 70s 211ms/step - loss: 0.0000e+00 - accuracy: 0.5017 - val_loss: 0.0000e+00 - val_accuracy: 0.4985
Epoch 3/50
333/333 [==============================] - 70s 211ms/step - loss: 0.0000e+00 - accuracy: 0.5017 - val_loss: 0.0000e+00 - val_accuracy: 0.4985
Epoch 4/50
333/333 [==============================] - 71s 215ms/step - loss: 0.0000e+00 - accuracy: 0.5017 - val_loss: 0.0000e+00 - val_accuracy: 0.4985
Epoch 5/50
333/333 [==============================] - 76s 229ms/step - loss: 0.0000e+00 - accuracy: 0.5017 - val_loss: 0.0000e+00 - val_accuracy: 0.4985
Epoch 6/50
 19/333 [>.............................] - ETA: 1:01 - loss: 0.0000e+00 - accuracy: 0.5230


KeyboardInterrupt

